<a href="https://colab.research.google.com/github/makonhakony/sms-text-classification/blob/main/Copy_of_fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers, losses
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.2/474.2 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.9/440.9 kB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 55.0 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.15.0
    Uninstalling wrapt-1.15.0:
      Successfully uninstalled wrapt-1.15.0
2.15.0-dev20231001


In [22]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
valid_file_path = "valid-data.tsv"

--2023-10-01 19:40:56--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 104.26.3.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv.1’

train-data.tsv.1    100%[===================>] 349.84K  --.-KB/s    in 0.04s   

2023-10-01 19:40:56 (7.81 MB/s) - ‘train-data.tsv.1’ saved [358233/358233]

--2023-10-01 19:40:56--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 104.26.3.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv.1’

valid-data.tsv.1    100%[========

# Data Investigation

In [23]:
import pandas as pd
column_names = ['label', 'text']
df_train = pd.read_csv(train_file_path,sep='\t', names=column_names)
df_valid = pd.read_csv(valid_file_path,sep='\t', names=column_names)

In [24]:
# IMPORTANT: 1 HAM 0 SPAM

df_train['label'] = df_train['label'].map({'ham': 1, 'spam': 0})
df_valid['label'] = df_valid['label'].map({'ham': 1, 'spam': 0})

In [82]:
df_train.tail()

,label,text
4174,1,just woke up. yeesh its late. but i didn't fal...
4175,1,what do u reckon as need 2 arrange transport i...
4176,0,free entry into our £250 weekly competition ju...
4177,0,-pls stop bootydelious (32/f) is inviting you ...
4178,1,tell my bad character which u dnt lik in me. ...


In [81]:
df_valid.tail()

,label,text
1387,1,true dear..i sat to pray evening and felt so.s...
1388,1,"what will we do in the shower, baby?"
1389,1,where are you ? what are you doing ? are yuou ...
1390,0,ur cash-balance is currently 500 pounds - to m...
1391,0,not heard from u4 a while. call 4 rude chat pr...


# **Test Vectorize**

Max text length

In [50]:
max = 0
for data in df_train['text']:
  if (max < len(data)):
    max = len(data)

print(max)

910


In [51]:
import re,string

def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
  return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation),
                                  '')

In [88]:
max_features = 10000
sequence_length = 910

vectorize_layer = keras.layers.TextVectorization()

In [89]:
#text only without labels

train_text = df_train['text'].values
vectorize_layer.adapt(train_text)

In [91]:
def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return vectorize_layer(text), label

In [92]:
print("1287 ---> ",vectorize_layer.get_vocabulary()[1287])
print(" 313 ---> ",vectorize_layer.get_vocabulary()[313])
print('Vocabulary size: {}'.format(len(vectorize_layer.get_vocabulary())))

1287 --->  530
 313 --->  into
Vocabulary size: 8181


In [98]:
# raw_train_ds = tf.data.Dataset.from_tensor_slices((df_train['label'].values,df_train['text'].values))
# raw_val_ds = tf.data.Dataset.from_tensor_slices((df_valid['label'].values,df_valid['text'].values))

In [97]:
# text_batch, label_batch = next(iter(raw_train_ds))
# first_review, first_label = text_batch[0], label_batch[0]
# print("Review", first_review)
# print("Label", raw_train_ds.class_names[first_label])
# print("Vectorized review", vectorize_text(first_review, first_label))

In [99]:
# train_ds = raw_train_ds.map(vectorize_text)
# val_ds = raw_val_ds.map(vectorize_text)

In [100]:
# AUTOTUNE = tf.data.AUTOTUNE
# train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
# val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

pre


Create Model

In [105]:
embedding_dim = 16

model = tf.keras.Sequential([
  vectorize_layer,
  layers.Embedding(
        input_dim=len(vectorize_layer.get_vocabulary()),
        output_dim=64,
        mask_zero=True),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid')
  ])

model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_7 (Text  (None, None)              0         
 Vectorization)                                                  
                                                                 
 embedding_11 (Embedding)    (None, None, 64)          523584    
                                                                 
 bidirectional_2 (Bidirecti  (None, 128)               66048     
 onal)                                                           
                                                                 
 dense_13 (Dense)            (None, 64)                8256      
                                                                 
 dense_14 (Dense)            (None, 1)                 65        
                                                                 
Total params: 597953 (2.28 MB)
Trainable params: 5979

In [106]:
model.compile(loss=losses.BinaryCrossentropy(),
              optimizer='adam',
              metrics=tf.metrics.BinaryAccuracy(threshold=0.0))

In [108]:
epochs = 10

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                  mode="min",
                                  patience=3,
                                  restore_best_weights=True)

history = model.fit(
    df_train['text'].values,
    df_train['label'].values,
    validation_data=(
        df_valid['text'].values,
        df_valid['label'].values
        ),
    epochs=epochs)

Epoch 1/10
131/131 [==============================] - 30s 141ms/step - loss: 0.2149 - binary_accuracy: 0.8660 - val_loss: 0.0533 - val_binary_accuracy: 0.8657
Epoch 2/10
131/131 [==============================] - 13s 102ms/step - loss: 0.0249 - binary_accuracy: 0.8660 - val_loss: 0.0404 - val_binary_accuracy: 0.8657
Epoch 3/10
131/131 [==============================] - 11s 83ms/step - loss: 0.0067 - binary_accuracy: 0.8660 - val_loss: 0.0368 - val_binary_accuracy: 0.8657
Epoch 4/10
131/131 [==============================] - 14s 107ms/step - loss: 0.0027 - binary_accuracy: 0.8660 - val_loss: 0.0432 - val_binary_accuracy: 0.8657
Epoch 5/10
131/131 [==============================] - 16s 123ms/step - loss: 5.5573e-04 - binary_accuracy: 0.8660 - val_loss: 0.0467 - val_binary_accuracy: 0.8657
Epoch 6/10
131/131 [==============================] - 15s 119ms/step - loss: 4.2421e-04 - binary_accuracy: 0.8660 - val_loss: 0.0498 - val_binary_accuracy: 0.8657
Epoch 7/10
131/131 [===================

In [113]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  res = model.predict(np.array([[pred_text]]))

  if res[0][0] >= 0.5:
    tag = 'ham'
  else:
    tag = 'spam'

  prediction = [res[0][0], tag]
  return (prediction)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

1/1 [==============================] - 0s 27ms/step
[0.999998, 'ham']


In [114]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 [==============================] - 0s 33ms/step
You passed the challenge. Great job!
